## Preprocesamiento de Datos de Acciones

Pipeline de preprocesamiento que implementa:
- Log-returns para High/Low/Open/Close
- Indicadores técnicos: RSI, MACD, Bandas de Bollinger, SMA
- Normalización de volumen
- Winsorización para outliers
- Características temporales cíclicas
- Múltiples estimadores de volatilidad

In [2]:
import os
from typing import List
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import talib as ta
import warnings
warnings.filterwarnings('ignore')

### Configuración

In [3]:
# Rutas de datos
DATA_PATH_LOAD = '../data/stocks/raw/'
DATA_PATH_SAVE = '../data/stocks/processed/'
os.makedirs(DATA_PATH_SAVE, exist_ok=True)

# Tickers a procesar
TICKERS = ['GOOGL', 'AAPL', 'AMZN', 'META', 'MSFT', 'NVDA', 'TSLA']

# Parámetros
VOLATILITY_WINDOW = 20
VOLUME_WINDOW = 20
WINSOR_LOWER_Q = 0.01
WINSOR_UPPER_Q = 0.99

REQUIRED_COLS = ['Open', 'High', 'Low', 'Close', 'Volume']

### Funciones Auxiliares

In [4]:
def ensure_numeric(df: pd.DataFrame, cols: List[str]):
    """Asegura que las columnas sean numéricas"""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    return df


def save_parquet(df: pd.DataFrame, ticker: str):
    """Guarda DataFrame como parquet"""
    file_path = os.path.join(DATA_PATH_SAVE, f"{ticker}_data_processed.parquet")
    df.to_parquet(file_path, index=False, compression='snappy')
    print(f"  -> Guardado: {file_path} (shape={df.shape})")

### Funciones de Features - Log Returns OHLC

In [5]:
def add_log_returns_ohlc(df: pd.DataFrame) -> pd.DataFrame:
    """Añade log returns para High/Low/Open/Close"""
    df = df.copy()
    for col in ['High', 'Low', 'Open', 'Close']:
        if col in df.columns:
            ratio = df[col] / df[col].shift(1)
            ratio = ratio.replace([np.inf, -np.inf], np.nan)
            ratio = ratio.where(ratio > 0, np.nan)
            df[f'LOG_RETURN_{col.upper()}'] = np.log(ratio)
    return df

### Medias Móviles y Bandas de Bollinger

In [6]:
def add_ma_and_bbands(df: pd.DataFrame) -> pd.DataFrame:
    """Añade medias móviles y Bandas de Bollinger"""
    df = df.copy()
    if 'Close' not in df.columns:
        return df

    # Medias móviles
    df['SMA_10'] = df['Close'].rolling(10, min_periods=1).mean()
    df['SMA_20'] = df['Close'].rolling(20, min_periods=1).mean()
    df['SMA_30'] = df['Close'].rolling(30, min_periods=1).mean()

    # Bandas de Bollinger (20, 2)
    rolling_mean = df['Close'].rolling(20, min_periods=1).mean()
    rolling_std = df['Close'].rolling(20, min_periods=1).std()
    df['MIDDLE_BAND'] = rolling_mean
    df['UPPER_BAND'] = rolling_mean + 2 * rolling_std
    df['LOWER_BAND'] = rolling_mean - 2 * rolling_std

    return df

### RSI (Relative Strength Index)

In [7]:
def add_rsi(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """Añade RSI"""
    df = df.copy()
    if 'Close' not in df.columns:
        return df

    if ta is not None:
        df[f'RSI_{period}'] = ta.RSI(df['Close'].values, timeperiod=period)
    else:
        # Implementación simple de RSI
        delta = df['Close'].diff()
        up = delta.clip(lower=0)
        down = -1 * delta.clip(upper=0)
        ma_up = up.rolling(window=period, min_periods=1).mean()
        ma_down = down.rolling(window=period, min_periods=1).mean()
        rs = ma_up / (ma_down.replace(0, np.nan))
        df[f'RSI_{period}'] = 100 - (100 / (1 + rs))
    return df

### MACD (Moving Average Convergence Divergence)

In [8]:
def add_macd(df: pd.DataFrame) -> pd.DataFrame:
    """Añade MACD, señal e histograma"""
    df = df.copy()
    if 'Close' not in df.columns:
        return df

    if ta is not None:
        macd, signal, hist = ta.MACD(df['Close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
        df['MACD'] = macd
        df['MACD_SIGNAL'] = signal
        df['MACD_HIST'] = hist
    else:
        # Fallback: MACD simple
        ema12 = df['Close'].ewm(span=12, adjust=False).mean()
        ema26 = df['Close'].ewm(span=26, adjust=False).mean()
        macd = ema12 - ema26
        signal = macd.ewm(span=9, adjust=False).mean()
        hist = macd - signal
        df['MACD'] = macd
        df['MACD_SIGNAL'] = signal
        df['MACD_HIST'] = hist
    return df

### Targets y Volatilidad

In [9]:
def add_targets_and_volatility(df: pd.DataFrame, vol_window: int = VOLATILITY_WINDOW) -> pd.DataFrame:
    """Añade log return principal, abs log return y volatilidad"""
    df = df.copy()
    # Log return principal (Close)
    ratio = df['Close'] / df['Close'].shift(1)
    df['LOG_RETURN'] = np.log(ratio)
    df['ABS_LOG_RETURN'] = df['LOG_RETURN'].abs()
    df['VOLATILITY'] = df['LOG_RETURN'].rolling(window=vol_window).std()
    return df

### Características Temporales Cíclicas

In [10]:
def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    """Añade características temporales cíclicas: día de semana, mes, trimestre, día del mes"""
    df = df.copy()
    if 'Date' not in df.columns:
        return df

    # Día de la semana (0=Lunes)
    df['DAY_OF_WEEK'] = df['Date'].dt.weekday
    df['DAY_OF_WEEK_SIN'] = np.sin(2 * np.pi * df['DAY_OF_WEEK'] / 7)
    df['DAY_OF_WEEK_COS'] = np.cos(2 * np.pi * df['DAY_OF_WEEK'] / 7)

    # Mes (1-12)
    df['MONTH'] = df['Date'].dt.month
    df['MONTH_SIN'] = np.sin(2 * np.pi * (df['MONTH'] - 1) / 12)
    df['MONTH_COS'] = np.cos(2 * np.pi * (df['MONTH'] - 1) / 12)

    # Día del mes (1-31)
    df['DAY_OF_MONTH'] = df['Date'].dt.day
    df['DAY_OF_MONTH_SIN'] = np.sin(2 * np.pi * (df['DAY_OF_MONTH'] - 1) / 31)
    df['DAY_OF_MONTH_COS'] = np.cos(2 * np.pi * (df['DAY_OF_MONTH'] - 1) / 31)

    # Trimestre (1-4)
    df['QUARTER'] = df['Date'].dt.quarter
    df['QUARTER_SIN'] = np.sin(2 * np.pi * (df['QUARTER'] - 1) / 4)
    df['QUARTER_COS'] = np.cos(2 * np.pi * (df['QUARTER'] - 1) / 4)

    return df

### Oscilador Estocástico

In [11]:
def add_stochastic_oscillator(df: pd.DataFrame, k_period: int = 14, d_period: int = 3) -> pd.DataFrame:
    """Añade Oscilador Estocástico (%K y %D)"""
    df = df.copy()
    if not all(col in df.columns for col in ['High', 'Low', 'Close']):
        return df
    
    # %K = 100 * (Close - Low14) / (High14 - Low14)
    low_min = df['Low'].rolling(window=k_period, min_periods=1).min()
    high_max = df['High'].rolling(window=k_period, min_periods=1).max()
    
    df['STOCH_K'] = 100 * (df['Close'] - low_min) / (high_max - low_min + 1e-10)
    
    # %D = SMA de 3 períodos de %K
    df['STOCH_D'] = df['STOCH_K'].rolling(window=d_period, min_periods=1).mean()
    
    return df

### Williams %R

In [12]:
def add_williams_r(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """Añade indicador Williams %R"""
    df = df.copy()
    if not all(col in df.columns for col in ['High', 'Low', 'Close']):
        return df
    
    # Williams %R = -100 * (Highest High - Close) / (Highest High - Lowest Low)
    high_max = df['High'].rolling(window=period, min_periods=1).max()
    low_min = df['Low'].rolling(window=period, min_periods=1).min()
    
    df['WILLIAMS_R'] = -100 * (high_max - df['Close']) / (high_max - low_min + 1e-10)
    
    return df

### Estimadores de Volatilidad

In [13]:
def add_realized_volatility(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """Añade Volatilidad Realizada (desviación estándar de log returns)"""
    df = df.copy()
    if 'LOG_RETURN' not in df.columns:
        return df
    
    df['REALIZED_VOL'] = df['LOG_RETURN'].rolling(window=window, min_periods=1).std()
    
    return df


def add_parkinson_volatility(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Añade estimador de Volatilidad de Parkinson
    Usa precios High y Low: σ = sqrt(1/(4*ln(2)) * mean((ln(H/L))^2))
    """
    df = df.copy()
    if not all(col in df.columns for col in ['High', 'Low']):
        return df
    
    hl_ratio = np.log(df['High'] / (df['Low'] + 1e-10))
    hl_ratio_sq = hl_ratio ** 2
    
    df['PARKINSON_VOL'] = np.sqrt(
        (1 / (4 * np.log(2))) * hl_ratio_sq.rolling(window=window, min_periods=1).mean()
    )
    
    return df


def add_garman_klass_volatility(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Añade estimador de Volatilidad de Garman-Klass
    GK = sqrt(0.5 * (ln(H/L))^2 - (2*ln(2)-1) * (ln(C/O))^2)
    """
    df = df.copy()
    if not all(col in df.columns for col in ['High', 'Low', 'Close', 'Open']):
        return df
    
    hl_ratio = np.log(df['High'] / (df['Low'] + 1e-10))
    co_ratio = np.log(df['Close'] / (df['Open'] + 1e-10))
    
    gk_component = 0.5 * (hl_ratio ** 2) - (2 * np.log(2) - 1) * (co_ratio ** 2)
    
    df['GARMAN_KLASS_VOL'] = np.sqrt(
        gk_component.rolling(window=window, min_periods=1).mean()
    )
    
    return df


def add_rogers_satchell_volatility(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Añade estimador de Volatilidad de Rogers-Satchell
    RS = sqrt(mean(ln(H/C) * ln(H/O) + ln(L/C) * ln(L/O)))
    """
    df = df.copy()
    if not all(col in df.columns for col in ['High', 'Low', 'Close', 'Open']):
        return df
    
    hc = np.log(df['High'] / (df['Close'] + 1e-10))
    ho = np.log(df['High'] / (df['Open'] + 1e-10))
    lc = np.log(df['Low'] / (df['Close'] + 1e-10))
    lo = np.log(df['Low'] / (df['Open'] + 1e-10))
    
    rs_component = hc * ho + lc * lo
    
    df['ROGERS_SATCHELL_VOL'] = np.sqrt(
        rs_component.rolling(window=window, min_periods=1).mean()
    )
    
    return df

### VWAP Estimado

In [14]:
def add_estimated_vwap(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Añade VWAP estimado (Volume Weighted Average Price)
    VWAP = sum(Precio * Volumen) / sum(Volumen)
    Usando precio típico: (High + Low + Close) / 3
    """
    df = df.copy()
    if not all(col in df.columns for col in ['High', 'Low', 'Close', 'Volume']):
        return df
    
    # Precio típico
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    
    # Cálculo de VWAP con ventana móvil
    df['VWAP'] = (
        (typical_price * df['Volume']).rolling(window=window, min_periods=1).sum() /
        df['Volume'].rolling(window=window, min_periods=1).sum()
    )
    
    return df

### Función Principal de Procesamiento

In [15]:
def process_stock(ticker: str) -> pd.DataFrame:
    """Procesa un ticker aplicando todas las transformaciones"""
    print('\n' + '=' * 60)
    print(f'Procesando {ticker}...')
    print('=' * 60)

    fp = os.path.join(DATA_PATH_LOAD, f"{ticker}_data.parquet")
    if not os.path.exists(fp):
        print(f'  ADVERTENCIA: archivo no encontrado: {fp}')
        return pd.DataFrame()

    df = pd.read_parquet(fp)
    
    # Asegurar que Date es una columna
    if df.columns.name is not None and df.columns.name == 'Price':
        df.columns.name = None

    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date').reset_index(drop=True)
    else:
        # Si Date es índice
        try:
            df = df.reset_index()
            if 'index' in df.columns:
                df = df.rename(columns={'index': 'Date'})
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values('Date').reset_index(drop=True)
        except Exception:
            pass

    # Asegurar tipos numéricos
    df = ensure_numeric(df, REQUIRED_COLS)

    # Añadir indicadores
    df = add_ma_and_bbands(df)
    df = add_rsi(df, period=14)
    df = add_macd(df)
    df = add_log_returns_ohlc(df)
    df = add_targets_and_volatility(df)
    df = add_temporal_features(df)
    
    # Indicadores adicionales
    df = add_stochastic_oscillator(df, k_period=14, d_period=3)
    df = add_williams_r(df, period=14)
    df = add_realized_volatility(df, window=20)
    df = add_parkinson_volatility(df, window=20)
    df = add_garman_klass_volatility(df, window=20)
    df = add_rogers_satchell_volatility(df, window=20)
    df = add_estimated_vwap(df, window=20)

    # Eliminar filas con NaN
    df = df.dropna().reset_index(drop=True)

    save_parquet(df, ticker)
    return df

### Procesar Todos los Tickers

In [16]:
processed = {}
for t in TICKERS:
    try:
        processed[t] = process_stock(t)
    except Exception as e:
        print(f'Error procesando {t}: {e}')
        continue

print('\n' + '='*60)
print('Resumen:')
print('='*60)
for t, df in processed.items():
    print(f'  {t}: {df.shape[0]} filas x {df.shape[1]} columnas')

print('\nProcesamiento completado.')


Procesando GOOGL...
  -> Guardado: ../data/stocks/processed/GOOGL_data_processed.parquet (shape=(2482, 43))

Procesando AAPL...
  -> Guardado: ../data/stocks/processed/AAPL_data_processed.parquet (shape=(2482, 43))

Procesando AMZN...
  -> Guardado: ../data/stocks/processed/AMZN_data_processed.parquet (shape=(2482, 43))

Procesando META...
  -> Guardado: ../data/stocks/processed/META_data_processed.parquet (shape=(2482, 43))

Procesando MSFT...
  -> Guardado: ../data/stocks/processed/MSFT_data_processed.parquet (shape=(2482, 43))

Procesando NVDA...
  -> Guardado: ../data/stocks/processed/NVDA_data_processed.parquet (shape=(2482, 43))

Procesando TSLA...
  -> Guardado: ../data/stocks/processed/TSLA_data_processed.parquet (shape=(2482, 43))

Resumen:
  GOOGL: 2482 filas x 43 columnas
  AAPL: 2482 filas x 43 columnas
  AMZN: 2482 filas x 43 columnas
  META: 2482 filas x 43 columnas
  MSFT: 2482 filas x 43 columnas
  NVDA: 2482 filas x 43 columnas
  TSLA: 2482 filas x 43 columnas

Proces

### Verificación de Datos Procesados

In [17]:
# Mostrar ejemplo de datos procesados
if processed:
    ticker_ejemplo = list(processed.keys())[0]
    df_ejemplo = processed[ticker_ejemplo]
    
    print(f"\nEjemplo: {ticker_ejemplo}")
    print(f"Shape: {df_ejemplo.shape}")
    print(f"\nColumnas generadas:")
    print(df_ejemplo.columns.tolist())
    print(f"\nPrimeras 5 filas:")
    display(df_ejemplo.head())
    
    print(f"\nÚltimas 5 filas:")
    display(df_ejemplo.tail())


Ejemplo: GOOGL
Shape: (2482, 43)

Columnas generadas:
['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_10', 'SMA_20', 'SMA_30', 'MIDDLE_BAND', 'UPPER_BAND', 'LOWER_BAND', 'RSI_14', 'MACD', 'MACD_SIGNAL', 'MACD_HIST', 'LOG_RETURN_HIGH', 'LOG_RETURN_LOW', 'LOG_RETURN_OPEN', 'LOG_RETURN_CLOSE', 'LOG_RETURN', 'ABS_LOG_RETURN', 'VOLATILITY', 'DAY_OF_WEEK', 'DAY_OF_WEEK_SIN', 'DAY_OF_WEEK_COS', 'MONTH', 'MONTH_SIN', 'MONTH_COS', 'DAY_OF_MONTH', 'DAY_OF_MONTH_SIN', 'DAY_OF_MONTH_COS', 'QUARTER', 'QUARTER_SIN', 'QUARTER_COS', 'STOCH_K', 'STOCH_D', 'WILLIAMS_R', 'REALIZED_VOL', 'PARKINSON_VOL', 'GARMAN_KLASS_VOL', 'ROGERS_SATCHELL_VOL', 'VWAP']

Primeras 5 filas:


,Date,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,MIDDLE_BAND,...,QUARTER_SIN,QUARTER_COS,STOCH_K,STOCH_D,WILLIAMS_R,REALIZED_VOL,PARKINSON_VOL,GARMAN_KLASS_VOL,ROGERS_SATCHELL_VOL,VWAP
0,2015-02-20,26.904440,27.191460,26.720708,27.191460,38234000,26.886561,26.564036,26.142063,26.564036,...,0.0,1.0,64.194595,74.827665,-35.805405,0.016117,0.012763,0.012712,0.012746,26.499567
1,2015-02-23,26.566769,26.780296,26.417796,26.765399,32906000,26.892122,26.546780,26.188559,26.546780,...,0.0,1.0,35.645470,60.343033,-64.354530,0.016247,0.012669,0.012578,0.012633,26.484452
2,2015-02-24,26.748016,26.785258,26.368135,26.395447,28406000,26.938652,26.551572,26.251342,26.551572,...,0.0,1.0,48.695030,49.511698,-51.304970,0.016182,0.012624,0.012441,0.012398,26.483352
3,2015-02-25,27.179045,27.290277,26.716235,26.737590,40836000,26.974256,26.616474,26.334552,26.616474,...,0.0,1.0,79.276456,54.538985,-20.723544,0.014956,0.012581,0.012464,0.012453,26.526638
4,2015-02-26,27.772949,27.814661,27.067812,27.100090,54032000,27.079977,26.732822,26.429712,26.732822,...,0.0,1.0,97.421750,75.131078,-2.578250,0.014809,0.012639,0.012534,0.012518,26.618415



Últimas 5 filas:


,Date,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,MIDDLE_BAND,...,QUARTER_SIN,QUARTER_COS,STOCH_K,STOCH_D,WILLIAMS_R,REALIZED_VOL,PARKINSON_VOL,GARMAN_KLASS_VOL,ROGERS_SATCHELL_VOL,VWAP
2477,2024-12-23,193.997543,194.466017,189.532090,191.994065,25675000,191.114929,180.934499,178.723773,180.934499,...,-1.0,-1.836970e-16,77.778433,68.723900,-22.221567,0.022009,0.016324,0.014932,0.014188,183.689147
2478,2024-12-24,195.472717,195.472717,193.150287,194.206840,10403300,192.205373,182.362440,179.254259,182.362440,...,-1.0,-1.836970e-16,82.155296,75.799125,-17.844704,0.021907,0.016222,0.014807,0.014084,184.759942
2479,2024-12-26,194.964386,196.110643,193.748349,194.515836,12046600,192.225308,183.691788,179.725653,183.691788,...,-1.0,-1.836970e-16,80.441436,80.125055,-19.558564,0.022031,0.016206,0.014830,0.014113,185.498168
2480,2024-12-27,192.133606,194.685300,190.030462,194.316492,18891400,192.305048,184.874120,180.193620,184.874120,...,-1.0,-1.836970e-16,68.815288,77.137340,-31.184712,0.022534,0.016487,0.015174,0.014506,186.233776
2481,2024-12-30,190.618546,191.924286,188.505424,189.183223,14264700,192.446585,185.994637,180.720601,185.994637,...,-1.0,-1.836970e-16,50.024515,66.427079,-49.975485,0.022697,0.016564,0.015258,0.014570,186.746240
